[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ic22YVZF74IiQy6cRDj-LwaOoM4BapVl?usp=sharing)



## Install dependencies




In [4]:
# !pip install transformers datasets evaluate accelerate peft
%pip install torch


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


## Dataset Preprocessing


In [1]:
import torch
from transformers import RobertaModel, RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model
from datasets import load_dataset, Dataset
import pandas as pd

/home/hice1/apadmanaban7/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [57]:
neologism_data = pd.read_csv('all_sampled_twitter.csv')
neologism_data['label'] = neologism_data['label'].astype("int64")

dataset = Dataset.from_pandas(neologism_data)

train_test_split = dataset.train_test_split(test_size=0.4, seed=42)
train_dataset = train_test_split["train"]
test_dataset_tmp = train_test_split["test"].train_test_split(test_size=0.5, seed=42)
eval_dataset = test_dataset_tmp["train"]
test_dataset = test_dataset_tmp["test"]

In [58]:
neologism_data

,text,label
0,"""@mattduss @chrislhayes if ISIS does gain a fo...",0
1,"""Sad news! 150 people shot, drowned in Yobe ri...",0
2,@StephenNolan they all served and died under t...,0
3,Rousey vs. Holm may be the biggest waste of a ...,0
4,@user Amy Schumer is the stereotypical 1st wor...,0
...,...,...
8739,The city experienced a complete blackout after...,0
8740,"After the power outage, the city experienced a...",0
8741,The unexpected blackout plunged the city into ...,0
8742,The company's sudden mobilization of resources...,0


In [60]:
peft_model_name = 'roberta-base-peft'
modified_base = 'roberta-base-modified'
base_model_tokenizer = 'roberta-base'
base_model = f'cardiffnlp/twitter-roberta-base-sentiment-latest'

tokenizer = RobertaTokenizer.from_pretrained(base_model_tokenizer)



def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)

train_dataset = train_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset = test_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset = eval_dataset.with_format("torch", columns=["input_ids", "attention_mask", "label"])

# tokenized_dataset = dataset.map(preprocess, batched=True,  remove_columns=["text"])
# train_dataset=tokenized_dataset['train']
# eval_dataset=tokenized_dataset['test'].shard(num_shards=2, index=0)
# test_dataset=tokenized_dataset['test'].shard(num_shards=2, index=1)


# Extract the number of classess and their names
num_labels = 3
class_names = ['negative', 'neutral', 'positive']
print(f"number of labels: {num_labels}")
print(f"the labels: {class_names}")

# Create an id2label mapping
# We will need this for our classifier.
id2label = {i: label for i, label in enumerate(class_names)}

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


Map: 100%|██████████| 1749/1749 [00:00<00:00, 2114.30 examples/s]

number of labels: 3
the labels: ['negative', 'neutral', 'positive']


In [61]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


## Training

Let's Train two models, one using LORA and the other with full fine-tuning.

Note the LORA training times and the number of trained parameters!

In [62]:
# use the same Training args for all models
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='steps',
    learning_rate=5e-5,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    fp16=torch.cuda.is_available()
)

In [63]:
def get_trainer(model):
      return  Trainer(
          model=model,
          args=training_args,
          train_dataset=train_dataset,
          eval_dataset=eval_dataset,
          data_collator=data_collator,
      )

In [64]:
full_finetuning_trainer = get_trainer(
    AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label, num_labels=num_labels),
)

full_finetuning_trainer.train()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TrainOutput(global_step=1640, training_loss=0.2803986156859049, metrics={'train_runtime': 278.292, 'train_samples_per_second': 94.254, 'train_steps_per_second': 5.893, 'total_flos': 6901464947005440.0, 'train_loss': 0.2803986156859049, 'epoch': 5.0})

In [65]:
print(train_dataset[0]["label"])

tensor(1)


### PEFT Training

In [66]:
model = AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label)

peft_config = LoraConfig(task_type="SEQ_CLS", inference_mode=False, r=8, lora_alpha=16, lora_dropout=0.1)
peft_model = get_peft_model(model, peft_config)

print('PEFT Model')
peft_model.print_trainable_parameters()

peft_lora_finetuning_trainer = get_trainer(peft_model)

peft_lora_finetuning_trainer.train()

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


PEFT Model
trainable params: 887,811 || all params: 125,535,750 || trainable%: 0.7072


Step,Training Loss,Validation Loss


TrainOutput(global_step=1640, training_loss=0.5189021599002, metrics={'train_runtime': 237.057, 'train_samples_per_second': 110.648, 'train_steps_per_second': 6.918, 'total_flos': 6973003478937600.0, 'train_loss': 0.5189021599002, 'epoch': 5.0})

In [67]:
# Save
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

## Performing Inference with a PEFT Model

It's time to have some fun putting our model to work!

In [70]:
from peft import AutoPeftModelForSequenceClassification
from transformers import AutoTokenizer


# LOAD the Saved PEFT model

inference_model = AutoPeftModelForSequenceClassification.from_pretrained(peft_model_name, id2label=id2label)
tokenizer = AutoTokenizer.from_pretrained(modified_base)


def classify(text):
  device = next(inference_model.parameters()).device  # Get device of model
  print(device)
  inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt").to("cuda")

  output = inference_model(**inputs)

  prediction = output.logits.argmax(dim=-1).item()

  print(f'\n Class: {prediction}, Label: {id2label[prediction]}, Text: {text}')
  # return id2label[prediction]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [78]:
classify( "Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia")
# classify( "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.")

cuda:0

 Class: 2, Label: positive, Text: Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia


### Evaluate Models

To measure the improvement of the Training process we will need a baseline; let's compare the trained models with an untrained one.

Take a Look how the trained models perform

1.   The trained models against the untrained one
2.   The PEFT Model vs the regular fine-tuned one  



In [ ]:
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

metric = evaluate.load('accuracy')
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def evaluate_model(inference_model, dataset):

    eval_dataloader = DataLoader(dataset.rename_column("label", "labels"), batch_size=8, collate_fn=data_collator)
    inference_model.to("cuda")
    inference_model.eval()
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch.to("cuda")
        with torch.no_grad():
            outputs = inference_model(**batch)
        predictions = outputs.logits.argmax(dim=-1)
        predictions, references = predictions, batch["labels"]
        metric.add_batch(
            predictions=predictions,
            references=references,
        )

    eval_metric = clf_metrics.compute()
    print(eval_metric)




In [73]:
# Evaluate the non fine-tuned model
evaluate_model(AutoModelForSequenceClassification.from_pretrained(base_model, id2label=id2label), test_dataset)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 219/219 [00:16<00:00, 13.34it/s]

{'accuracy': 0.7695826186392224}


In [74]:
# Evaluate the PEFT fine-tuned model
evaluate_model(inference_model, test_dataset)

100%|██████████| 219/219 [00:17<00:00, 12.73it/s]

{'accuracy': 0.805603201829617}


In [75]:
# Evaluate the Fully fine-tuned model
evaluate_model(full_finetuning_trainer.model, test_dataset)

100%|██████████| 219/219 [00:05<00:00, 41.15it/s]

{'accuracy': 0.7753001715265866}


In [80]:
samples = [    "Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia",
        "That fit is straight fire, no cap, you're looking mad schmick",
        "The candidate's speech was pure clickbait, all sizzle and no steak",
        "The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.",
        "Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.",
        "That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the point of absurdity",
        "Taylor's new album slaps hard"]
        

for i in samples:
    print(classify(i))

cuda:0

 Class: 2, Label: positive, Text: Listening to the retro playlist filled with 80's synth-pop hits, he was filled by a wave of falstalagia
None
cuda:0

 Class: 2, Label: positive, Text: That fit is straight fire, no cap, you're looking mad schmick
None
cuda:0

 Class: 0, Label: negative, Text: The candidate's speech was pure clickbait, all sizzle and no steak
None
cuda:0

 Class: 2, Label: positive, Text: The workshop was absolute chaos, with ideas flying in every direction, but somehow, in the middle of that whirlwind, we created something ridiculously inspiring.
None
cuda:0

 Class: 2, Label: positive, Text: Absolutely love how our talent pool is just bursting with sparkle-genius nepo babies, each one more effortlessly deserving and glow-ready for success than anyone who actually worked for it.
None
cuda:0

 Class: 0, Label: negative, Text: That team-building session was an absolute trainwreck of synergy—everyone was crushed by inspiration and overloaded with motivation to the

In [10]:
dd["train"].features["label"].names

['World', 'Sports', 'Business', 'Sci/Tech']